In [28]:
# Обеспечим совместимость с Python 2 и 3
# pip install future
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)

# отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

# импортируем Pandas и Numpy
import pandas as pd
import numpy as np
df = pd.read_csv('X_train.csv')
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 1000)
df.property = str(df.property)
print(df.info())
df.head()


ValueError: array is too big; `arr.size * arr.dtype.itemsize` is larger than the maximum possible size.

Всё нужное импортировали, можно писать код